In [1]:
from datetime import date
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import matplotlib.animation as animation
warnings.filterwarnings('ignore')
from src.data_processing.fetch_data import fetch_data_subgraph
from src.models.liquidity_distribution import LiquidityAnalyzer, Pool, LiquidityPeriphery
pd.set_option("display.max_columns",100)
%load_ext autoreload
%autoreload 2

POOLS = [
    (3, "0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8", "v3 USDC/ETH 0.3%", ),
    (3, "0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640", "v3 USDC/ETH 0.05%", ),
]
API_KEY = "893d7471304c5edf436c8ba60781762c"
POOL_ADDRESS = "0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640".lower()
START_DATE = date(2024, 8, 1)
END_DATE = date(2024, 9, 1)
pool_query = """query get_pools($pool_id: ID!) {
  pools(where: {id: $pool_id}) {
    tick
    sqrtPrice
    liquidity
    feeTier
    totalValueLockedUSD
    totalValueLockedETH
    token0 {
      symbol
      decimals
    }
    token1 {
      symbol
      decimals
    }
  }
}"""
# POOL_ADDRESS = "0xc7bBeC68d12a0d1830360F8Ec58fA599bA1b0e9b".lower()
variables = {'pool_id': POOL_ADDRESS}
data_pool = fetch_data_subgraph(API_KEY, pool_query, variables, data_key='pools', first_n=100, batch_size=1000)[0]
pool = Pool(pool_address=POOL_ADDRESS,
            token0=data_pool['token0']['symbol'],
            token1=data_pool['token1']['symbol'],
            decimals0=data_pool['token0']['decimals'],
            decimals1=data_pool['token1']['decimals'],
            fee_tier=data_pool['feeTier'],
            sqrt_price_x96=data_pool["sqrtPrice"])

analyzer = LiquidityAnalyzer(
    pool,
    start_date=START_DATE,
    end_date=END_DATE,
    threshold_dollar_volume_swaps=20000000
)

Fetched 1 records
Loading liquidity for block number 18909995
Loading liquidity for block number 18911563
Loading liquidity for block number 18912951
Loading liquidity for block number 18913979
Loading liquidity for block number 18914758
Loading liquidity for block number 18915118
Loading liquidity for block number 18915921
Loading liquidity for block number 18916179
Loading liquidity for block number 18916530
Loading liquidity for block number 18917177
Loading liquidity for block number 18917970
Loading liquidity for block number 18918719
Loading liquidity for block number 18919672
Loading liquidity for block number 18920396
Loading liquidity for block number 18920664
Loading liquidity for block number 18921197
Loading liquidity for block number 18921826
Loading liquidity for block number 18922278
Loading liquidity for block number 18923089
Loading liquidity for block number 18923952
Loading liquidity for block number 18925628
Loading liquidity for block number 18926513
Loading liquid

In [10]:
def subselect_active_liquidity(nr_tick_around):
       analyzer.df_liquidity.columns = ['block', 'active_tick', 'tick', 'liquidity', 'price', 'amount0',
              'amount1', 'amount0_real', 'amount1_real']
       liquidity = pd.merge(analyzer.sampled_blocks,
              analyzer.df_liquidity,
              left_on='block_number',
              right_on='block')
       liquidity['active_tick_adjusted'] = np.floor(liquidity['active_tick'] / pool.tick_spacing) * pool.tick_spacing
       liquidity['sb'] = liquidity['tick']-pool.tick_spacing*nr_tick_around
       liquidity['sa'] = liquidity['tick']+pool.tick_spacing*nr_tick_around
       liquidity.loc[liquidity['tick']<liquidity['active_tick_adjusted'],'amount0_real'] = 0
       liquidity.loc[liquidity['tick']>liquidity['active_tick_adjusted'],'amount1_real'] = 0
       liquidity['active_price'] = np.float64(liquidity['active_tick'].apply(lambda x: pool.adjust_price(pool.tick_to_price(x))))
       liquidity['active_price_inverted'] = 1/liquidity['active_price']
       liquidity['amount_locked'] = liquidity['amount1_real']*liquidity['active_price_inverted'] + liquidity['amount0_real']
       liquidity = liquidity.loc[liquidity['amount_locked']<50000000]
       active_liquidity = liquidity[(liquidity['active_tick_adjusted'] >= liquidity['sb']) & (liquidity['active_tick_adjusted'] <= liquidity['sa'])]
       active_liquidity = active_liquidity.reset_index(drop=True)
       return active_liquidity, liquidity

active_liquidity, liquidity_data = subselect_active_liquidity(50)
active_liquidity['amount_locked_USDC'] = active_liquidity.groupby(by='block_number')['amount_locked'].transform('sum')
active_liquidity_trading = active_liquidity[['datetime','block_number','active_tick_adjusted','active_price_inverted','cumulative_dollar_volume','amount_locked_USDC']].drop_duplicates()
active_liquidity_trading['trading_volume'] = active_liquidity_trading['cumulative_dollar_volume'] - active_liquidity_trading['cumulative_dollar_volume'].shift()
active_liquidity_trading['trading_fees'] = active_liquidity_trading['trading_volume'] * (pool.fee_tier / 1e6)
active_liquidity_trading['ROI_pool'] = active_liquidity_trading['trading_fees']/active_liquidity_trading['amount_locked_USDC']
active_liquidity_trading['date'] = active_liquidity_trading['datetime'].dt.normalize() 

In [13]:
active_liquidity.to_parquet("/Users/gnapsamuel/Documents/AMM/proteus-LP-backtesting/data/liquidity_data.parquet")

In [47]:
example = active_liquidity.loc[active_liquidity['block_number']==20773531]

In [152]:
example.head(2)

,block_number,block_timestamp,cumulative_dollar_volume,date,datetime,block,active_tick,tick,liquidity,price,amount0,amount1,amount0_real,amount1_real,active_tick_adjusted,sb,sa,active_price,active_price_inverted,amount_locked,amount_locked_USDC
348767,20773531,1726614827,7.016129e+10,2024-09-17,2024-09-17 23:13:47,20773531,198789,198280,7.955235e+18,2450.414443,1.968396e+11,8.036929e+19,0.0,80.369289,198780.0,197780,198780,0.000429,2328.815375,187165.235411,1.878030e+07
348768,20773531,1726614827,7.016129e+10,2024-09-17,2024-09-17 23:13:47,20773531,198789,198290,7.954941e+18,2447.965376,1.967340e+11,8.040651e+19,0.0,80.406510,198780.0,197790,198790,0.000429,2328.815375,187251.916073,1.878030e+07


In [14]:
df_liquidity = pd.read_parquet(
        "/Users/gnapsamuel/Documents/AMM/proteus-LP-backtesting/data/liquidity_data.parquet"
    )

In [20]:
df_liquidity.iloc[0]['active_price_inverted']

block_number                              18909995
block_timestamp                         1704080591
cumulative_dollar_volume           20028044.555969
date                                    2024-01-01
datetime                       2024-01-01 03:43:11
block                                     18909995
active_tick                                 199011
tick                                        198510
liquidity                    7536409291119563776.0
price                                  2394.700859
amount0                        184344352873.256287
amount1                     77018614993131192320.0
amount0_real                                   0.0
amount1_real                             77.018615
active_tick_adjusted                      199010.0
sb                                          198010
sa                                          199010
active_price                              0.000439
active_price_inverted                  2277.687846
amount_locked                  